In [1]:
# Instalación de dependencias
!pip install transformers torch sentencepiece accelerate -q

In [9]:
import torch
from transformers import pipeline

In [10]:
print("🩺 CARGANDO MODELO BIOBERT PARA NER MÉDICO...")
print("⏳ Esto puede tomar unos segundos...")

# Cargar modelo específico para NER médico
try:
    medical_ner = pipeline(
        "ner",
        model="d4data/biomedical-ner-all",  # Modelo especializado en NER médico
        aggregation_strategy="simple",
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Modelo médico especializado cargado exitosamente!")
    print(f"💻 Usando: {'GPU' if torch.cuda.is_available() else 'CPU'}")

except Exception as e:
    print(f"❌ Error cargando modelo principal: {e}")
    print("🔄 Cargando modelo alternativo...")

    try:
        medical_ner = pipeline(
            "ner",
            model="samrawal/bert-base-uncased_clinical-ner",
            aggregation_strategy="simple",
            device=0 if torch.cuda.is_available() else -1
        )
        print("✅ Modelo clínico alternativo cargado.")
    except Exception as e2:
        print(f"❌ Error con modelo alternativo: {e2}")
        medical_ner = None

🩺 CARGANDO MODELO BIOBERT PARA NER MÉDICO...
⏳ Esto puede tomar unos segundos...


Device set to use cpu


✅ Modelo médico especializado cargado exitosamente!
💻 Usando: CPU


In [11]:
def analizar_texto_medico(texto):
    """Analiza texto médico y extrae entidades con formato claro"""
    if not texto.strip():
        return "❌ Texto vacío. Por favor ingrese texto médico."

    if medical_ner is None:
        return "❌ Modelo no disponible."

    try:
        print("\n🔍 Analizando texto...")
        entidades = medical_ner(texto)

        if not entidades:
            return "📭 No se encontraron entidades médicas reconocibles."

        # Organizar resultados por tipo de entidad
        resultado = "\n" + "="*70
        resultado += "\n🩺 **RESULTADOS DEL ANÁLISIS NER MÉDICO:**"
        resultado += "\n" + "="*70

        # Agrupar entidades por tipo
        entidades_por_tipo = {}
        for entidad in entidades:
            tipo = entidad['entity_group']
            if tipo not in entidades_por_tipo:
                entidades_por_tipo[tipo] = []
            entidades_por_tipo[tipo].append(entidad)

        # Mostrar resumen
        total_entidades = len(entidades)
        resultado += f"\n\n📊 RESUMEN: {total_entidades} entidades encontradas"
        resultado += "\n" + "-"*40

        for tipo, lista in entidades_por_tipo.items():
            resultado += f"\n   • {tipo}: {len(lista)} entidades"

        # Mostrar detalle organizado por tipo
        resultado += "\n\n📋 DETALLE POR CATEGORÍAS:"
        resultado += "\n" + "-"*40

        for tipo, lista_entidades in entidades_por_tipo.items():
            resultado += f"\n\n🏷️  **{tipo.upper()}**:"
            for i, entidad in enumerate(lista_entidades, 1):
                confianza = entidad['score']
                # Color de confianza
                if confianza > 0.9:
                    confianza_str = f"🟢 {confianza:.3f}"
                elif confianza > 0.7:
                    confianza_str = f"🟡 {confianza:.3f}"
                else:
                    confianza_str = f"🔴 {confianza:.3f}"

                resultado += f"\n   {i}. {entidad['word']}"
                resultado += f"\n       └─ Confianza: {confianza_str}"
                resultado += f"\n       └─ Posición: {entidad['start']}-{entidad['end']}"

        resultado += "\n\n" + "="*70
        resultado += "\n💡 **LEYENDA:** 🟢 Alta confianza (>0.9) | 🟡 Media confianza (0.7-0.9) | 🔴 Baja confianza (<0.7)"
        resultado += "\n" + "="*70

        return resultado

    except Exception as e:
        return f"❌ Error en el análisis: {str(e)}"


In [12]:
def mostrar_ejemplos():
    """Muestra ejemplos de texto médico para probar"""
    ejemplos = {
        "inglés": [
            "A 58-year-old male with Type 2 diabetes mellitus and hypertension was prescribed metformin 500mg and lisinopril 10mg for blood pressure control.",
            "Patient presented with acute myocardial infarction. ECG showed ST elevation. Treatment included aspirin and clopidogrel.",
            "Breast cancer diagnosis with ER-positive status. Treatment plan includes tamoxifen and radiation therapy."
        ],
        "español": [
            "Paciente con diabetes mellitus tipo 2 e hipertensión arterial. Tratamiento: metformina 500mg y losartán 50mg.",
            "Diagnóstico de neumonía bacteriana. Tratamiento con amoxicilina 500mg cada 8 horas.",
            "Paciente con enfermedad de Alzheimer, muestra deterioro cognitivo progresivo."
        ]
    }

    return ejemplos

# INTERFAZ PRINCIPAL POR TERMINAL
def interfaz_principal():
    print("\n" + "🌟" * 30)
    print("🌟    BIOBERT - ANALIZADOR MÉDICO NER    🌟")
    print("🌟" * 30)
    print("\nℹ️  Este sistema identifica entidades médicas usando IA")
    print("   Entidades: Enfermedades, Medicamentos, Síntomas, etc.\n")

    ejemplos = mostrar_ejemplos()

    while True:
        print("\n" + "="*60)
        print("MENÚ PRINCIPAL:")
        print("="*60)
        print("1. 📝 Analizar texto personalizado")
        print("2. 🇺🇸 Ejemplos en inglés")
        print("3. 🇪🇸 Ejemplos en español")
        print("4. ℹ️  Información del modelo")
        print("5. ❌ Salir")
        print("="*60)

        opcion = input("\nSeleccione una opción (1-5): ").strip()

        if opcion == "1":
            print("\n📝 MODO: TEXTO PERSONALIZADO")
            print("-" * 40)
            print("Ingrese el texto médico a analizar:")
            print("(Presione Enter dos veces para finalizar la entrada)")

            lineas = []
            while True:
                linea = input()
                if linea == "" and lineas:  # Doble Enter para terminar
                    break
                if linea == "" and not lineas:  # Evitar terminar con entrada vacía
                    continue
                lineas.append(linea)

            texto_usuario = "\n".join(lineas)

            if texto_usuario.strip():
                print("\n⏳ Procesando...")
                resultado = analizar_texto_medico(texto_usuario)
                print(resultado)

                # Preguntar si quiere guardar resultado
                guardar = input("\n¿Desea guardar este resultado en un archivo? (s/n): ").lower()
                if guardar == 's':
                    with open("resultado_ner_medico.txt", "w", encoding="utf-8") as f:
                        f.write(resultado)
                    print("💾 Resultado guardado en 'resultado_ner_medico.txt'")
            else:
                print("❌ No se ingresó texto válido.")

        elif opcion == "2":
            print("\n🇺🇸 EJEMPLOS EN INGLÉS (Recomendado):")
            print("-" * 50)
            for i, ejemplo in enumerate(ejemplos["inglés"], 1):
                print(f"\n{i}. {ejemplo}")

            seleccion = input(f"\nSeleccione ejemplo (1-{len(ejemplos['inglés'])}): ").strip()
            try:
                idx = int(seleccion) - 1
                if 0 <= idx < len(ejemplos["inglés"]):
                    print(f"\n📄 Ejemplo seleccionado:")
                    print(f"{ejemplos['inglés'][idx]}")
                    resultado = analizar_texto_medico(ejemplos["inglés"][idx])
                    print(resultado)
                else:
                    print("❌ Selección inválida.")
            except ValueError:
                print("❌ Ingrese un número válido.")

        elif opcion == "3":
            print("\n🇪🇸 EJEMPLOS EN ESPAÑOL:")
            print("-" * 50)
            for i, ejemplo in enumerate(ejemplos["español"], 1):
                print(f"\n{i}. {ejemplo}")

            seleccion = input(f"\nSeleccione ejemplo (1-{len(ejemplos['español'])}): ").strip()
            try:
                idx = int(seleccion) - 1
                if 0 <= idx < len(ejemplos["español"]):
                    print(f"\n📄 Ejemplo seleccionado:")
                    print(f"{ejemplos['español'][idx]}")
                    resultado = analizar_texto_medico(ejemplos["español"][idx])
                    print(resultado)
                else:
                    print("❌ Selección inválida.")
            except ValueError:
                print("❌ Ingrese un número válido.")

        elif opcion == "4":
            print("\nℹ️  INFORMACIÓN DEL MODELO:")
            print("-" * 40)
            print("• Modelo: BioBERT fine-tuned para NER médico")
            print("• Entidades identificadas:")
            print("  - Enfermedades y condiciones")
            print("  - Medicamentos y drogas")
            print("  - Síntomas y signos")
            print("  - Procedimientos médicos")
            print("  - Anatomía y órganos")
            print("  - Sustancias químicas")
            print("\n💡 Recomendación: Use texto en inglés para mejores resultados")

        elif opcion == "5":
            print("\n👋 ¡Gracias por usar el Analizador Médico NER!")
            break

        else:
            print("❌ Opción no válida. Por favor seleccione 1-5.")

        # Pausa antes de volver al menú
        if opcion in ["1", "2", "3"]:
            input("\nPresione Enter para continuar...")


In [13]:
# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    try:
        interfaz_principal()
    except KeyboardInterrupt:
        print("\n\n⚠️  Programa interrumpido por el usuario.")
    except Exception as e:
        print(f"\n❌ Error inesperado: {e}")

    print("\n" + "🎯" * 30)
    print("🎯        PROGRAMA TERMINADO        🎯")
    print("🎯" * 30)


🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
🌟    BIOBERT - ANALIZADOR MÉDICO NER    🌟
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟

ℹ️  Este sistema identifica entidades médicas usando IA
   Entidades: Enfermedades, Medicamentos, Síntomas, etc.


MENÚ PRINCIPAL:
1. 📝 Analizar texto personalizado
2. 🇺🇸 Ejemplos en inglés
3. 🇪🇸 Ejemplos en español
4. ℹ️  Información del modelo
5. ❌ Salir

Seleccione una opción (1-5): 2

🇺🇸 EJEMPLOS EN INGLÉS (Recomendado):
--------------------------------------------------

1. A 58-year-old male with Type 2 diabetes mellitus and hypertension was prescribed metformin 500mg and lisinopril 10mg for blood pressure control.

2. Patient presented with acute myocardial infarction. ECG showed ST elevation. Treatment included aspirin and clopidogrel.

3. Breast cancer diagnosis with ER-positive status. Treatment plan includes tamoxifen and radiation therapy.

Seleccione ejemplo (1-3): 1

📄 Ejemplo seleccionado:
A 58-year-old male with Type 2 diabetes mellitus and hypertension was prescrib